In [10]:
import csv
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

sk = []
kr = []
uid = []
peak = []
duration = []
freq = []
high_avg = []

df = pd.read_csv('training_set_0s_60.csv')
df = df.append(pd.read_csv('training_set_1s_60.csv'))
#print(df)

df = df.groupby('id', as_index=False, sort=False)['ev', 'skewness', 'kurtosis', 'mean_peak', 
                                                  'avg_high_energy_usage_duration', 
                                                  'high_energy_usage_frequency', 'mean_consumption'].mean()
#testing...
df.to_csv('aggregate_training_60min.csv')
uid = np.array(df['ev'].values)
sk = np.array(df['skewness'].values)
kr = np.array(df['kurtosis'].values)
peak = np.array(df['mean_peak'].values)
duration = np.array(df['avg_high_energy_usage_duration'].values)
freq = np.array(df['high_energy_usage_frequency'].values)
consumption_avg = np.array(df['mean_consumption'].values)

#normalization
#sk_norm = sk/np.linalg.norm(sk, ord=np.inf, axis=0, keepdims=True)
#kr_norm = kr/np.linalg.norm(kr, ord=np.inf, axis=0, keepdims=True)
#peak_norm = peak/np.linalg.norm(peak, ord=np.inf, axis=0, keepdims=True)
#duration_norm = duration/np.linalg.norm(duration, ord=np.inf, axis=0, keepdims=True)
#freq_norm = freq/np.linalg.norm(freq, ord=np.inf, axis=0, keepdims=True)
#high_avg_norm = high_avg/np.linalg.norm(high_avg, ord=np.inf, axis=0, keepdims=True)
#kr_norm = sklearn.preprocessing.normalize(kr)

X = []
with open('training_set_norm.csv', 'w') as fou:
    #w = csv.writer(fou, delimiter=',', lineterminator='\n')
    #w.writerow(['ev_id', 'skew_norm', 'kurtosis_norm', 'peak_norm', 'duration_norm', 'freq_norm', 'high_avg_norm'])
    #for i in range(0, len(uid)):
    #    X.append([sk_norm[i], kr_norm[i], high_avg_norm[i], peak_norm[i]])
    #    w.writerow([uid[i], sk_norm[i], kr_norm[i], peak_norm[i], duration_norm[i], freq_norm[i], high_avg_norm[i]])
    w = csv.writer(fou, delimiter=',', lineterminator='\n')
    w.writerow(['ev_id', 'skew_norm', 'kurtosis_norm', 'high_avg_norm', 'duration_norm', 'freq_norm', 'mean_consumption'])
    for i in range(0, len(uid)):
        X.append([sk[i], kr[i], peak[i], duration[i], freq[i], consumption_avg[i]])
        w.writerow([uid[i], sk[i], kr[i], peak[i], duration[i], freq[i], consumption_avg[i]])
#, peak_norm[i],duration_norm[i],freq_norm[i],high_avg_norm[i]        

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


Y = uid
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
#X_train = X
#X_test = X
#Y_train = Y
#Y_test = Y
from sklearn import preprocessing
from joblib import dump, load
scaler = preprocessing.StandardScaler().fit(X_train)
dump(scaler, 'scaler_60.joblib')
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)
#print(X_train_transformed, Y_train)
clf1 = SVC(gamma='scale', class_weight='balanced', kernel='rbf', shrinking=True, probability=True)
#print(Y)
clf1.fit(X_train_transformed, Y_train)
#print(clf.support_vectors_)
#print(clf.n_support_) 

dump(clf1, 'trained_ev_detect_svm_60.joblib') 
test_Y = clf1.predict(X_test_transformed)

#clf = LogisticRegression(random_state=0, solver='sag').fit(X, Y)
#test_X = clf.predict(X)
false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('svm') 
print(clf1.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

#clf2 = RandomForestClassifier(random_state=1)
clf3 = LogisticRegression(penalty='l2', solver='lbfgs', random_state=1, class_weight='balanced')
#eclf = VotingClassifier(estimators=[('svc', clf1), ('lr', clf3)], voting='soft', weights=[2, 1])
#clf2.fit(X, Y)
clf3.fit(X_train_transformed, Y_train)
dump(clf3, 'trained_ev_detect_lr_60.joblib') 
test_Y = clf3.predict(X_test_transformed)

#eclf.fit(X, Y)
#test_Y_el = eclf.predict(X)
false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('logregression') 
print(clf3.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)


clf2 = RandomForestClassifier(n_estimators=200, min_samples_split=3, min_samples_leaf=2, max_depth=25)

clf2.fit(X_train_transformed, Y_train)
test_Y = clf2.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('randomforest')
score_proba = clf2.predict_proba(X_test_transformed)
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test, score_proba[:,1])
print(tpr)
print(fpr)
print(thresholds)
print(clf2.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)
from sklearn import metrics
print(metrics.auc(fpr, tpr))
dump(clf2, 'trained_ev_detect_rf_60.joblib')

dt_stump = DecisionTreeClassifier(max_depth=10, min_samples_leaf=10)
clf4 = AdaBoostClassifier(base_estimator=dt_stump)
clf4.fit(X_train_transformed, Y_train)
test_Y = clf4.predict(X_test_transformed)

false_positive = 0
false_negative = 0 
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('adaboost') 
print(clf4.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)
dump(clf4, 'trained_ev_detect_adaboost_60.joblib')

from sklearn.ensemble import GradientBoostingClassifier
clf5 = GradientBoostingClassifier(n_estimators=150, min_samples_split=3, min_samples_leaf=2, 
                                  max_depth=25, random_state=0)
clf5.fit(X_train_transformed, Y_train)
test_Y = clf5.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('gradientboost')  
print(clf5.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

from sklearn.neighbors import KNeighborsClassifier
clf6 = KNeighborsClassifier(n_neighbors=2)
clf6.fit(X_train_transformed, Y_train)
test_Y = clf6.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('KNN')  
print(clf5.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

eclf = VotingClassifier(estimators=[('lr', clf3), ('gdb', clf5), ('rf', clf2)], 
                        voting='soft', weights=[1,2,1])
eclf.fit(X_train_transformed, Y_train)
test_Y = eclf.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
            
print(false_positive, true_negative, false_negative, true_positive)
print(eclf.score(X_test_transformed, Y_test))
score_proba = eclf.predict_proba(X_test_transformed)
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test, score_proba[:,1])
print(tpr)
print(fpr)
print(thresholds)
print(eclf.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)
from sklearn import metrics
print(metrics.auc(fpr, tpr))
dump(eclf, 'trained_ev_detect_ensemble_60.joblib') 

svm
0.8
0 20 12 28
logregression
0.8
0 20 12 28
randomforest
[0.    0.175 0.725 0.725 0.775 0.775 0.8   0.8   0.825 0.825 0.875 0.875
 0.975 0.975 1.    1.   ]
[0.   0.   0.   0.05 0.05 0.15 0.15 0.2  0.2  0.3  0.3  0.4  0.4  0.7
 0.7  1.  ]
[2.         1.         0.71559524 0.712      0.47309524 0.44975
 0.43254167 0.41109524 0.34667857 0.3157619  0.29683333 0.27190476
 0.23955952 0.17425    0.1654944  0.08734524]
0.8
1 19 11 29
0.91625
adaboost
0.8333333333333334
1 19 9 31
gradientboost
0.8
1 19 11 29
KNN
0.8
3 17 15 25
1 19 11 29
0.8
[0.    0.025 0.7   0.7   0.775 0.775 0.825 0.825 0.85  0.85  0.9   0.9
 0.925 0.925 0.95  0.95  0.975 0.975 1.    1.   ]
[0.   0.   0.   0.05 0.05 0.1  0.1  0.2  0.2  0.25 0.25 0.35 0.35 0.4
 0.4  0.5  0.5  0.8  0.8  1.  ]
[1.99996975 0.99996975 0.82555252 0.71318668 0.34422492 0.289722
 0.26620506 0.20025278 0.1927234  0.17576157 0.15541281 0.15087119
 0.14863647 0.13965208 0.13638762 0.13196608 0.1301221  0.1042392
 0.10261439 0.08538743]
0.8
1 19 11 

['trained_ev_detect_ensemble_60.joblib']